## DATA

In [1]:
import pymysql
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import font_manager, rc
import matplotlib
import os

In [2]:
con = pymysql.connect(host='192.168.0.150', user= 'jin', passwd='tiger', db='final_project', charset='utf8',autocommit=True)

In [3]:
cur=con.cursor(pymysql.cursors.DictCursor)
data = pd.read_sql('select * from animal_list', con)

In [4]:
#cur=con.cursor(pymysql.cursors.DictCursor)
#pred_set = pd.read_sql("select * from animalinfo3 where status like '보호중'", con)

In [5]:
data.columns

Index(['regnum', 'noticeend', 'age', 'color', 'animal', 'species', 'neuteryn',
       'sex', 'feature', 'weight', 'status', 'outer_health', 'inner_health',
       'disable', 'personality', 'care'],
      dtype='object')

In [ ]:
aa = data.drop(['regnum', 'noticeend', 'animal', 'feature', 'weight'], axis=1)

In [ ]:
aa.info()

In [ ]:
aa =aa.fillna(0)

In [ ]:
aa.describe()

In [ ]:
#can't display korean >>eng
#보호중, 종료(기증), 종료(안락사), 종료(입양), 종료(자연사)
aa.loc[aa['status'] =='보호중', 'status'] = 'protect'
aa.loc[aa['status'] =='종료(기증)', 'status'] = 'adopt'
aa.loc[aa['status'] =='종료(안락사)', 'status'] = 'death'
aa.loc[aa['status'] =='종료(입양)', 'status'] = 'adopt'
aa.loc[aa['status'] =='종료(자연사)', 'status'] = 'death'

In [ ]:
#객체에 status가 보호중인 index를 저장
drop_adopt = aa[aa['status'] == 'protect'].index

In [ ]:
bb = aa.drop(drop_adopt)

In [ ]:
bb.info()

In [ ]:
for i in bb.columns:
    print(i)
    bb[i].value_counts().plot(kind = 'bar')
    plt.show()

## preprocessing

y : adopt=1, others=0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
y = bb['status']
x = bb.drop('status', axis=1)

In [ ]:
x.shape, y.shape

In [ ]:
y.value_counts()

In [ ]:
bb.columns

In [ ]:
x_1 = pd.get_dummies(x,columns=['color', 'species', 'sex','neuteryn','outer_health','inner_health', 'disable', 'personality', 'care']
                          ,drop_first=True)

#xtest = pd.get_dummies(xtest,columns=['color', 'species', 'sex','neuteryn','outer_health','inner_health', 'disable', 'personality', 'care']
 #                         ,drop_first=True)
#ytrain_1 = pd.get_dummies(ytrain, drop_first = True)
#ytest_1 = pd.get_dummies(ytest, drop_first = True)

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x_1,y, test_size=0.3)

In [ ]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

## Logistic Regression

In [ ]:
logis = LogisticRegression()

In [ ]:
grid_values = {'C': [0.001,0.01,0.1,1,10,100,1000], 'max_iter':[1000, 2000, 3000, 5000]}
gscv = GridSearchCV(logis, param_grid=grid_values, return_train_score=True)

In [ ]:
gscv.fit(xtrain, ytrain)

In [ ]:
gscv.best_params_, gscv.best_index_, gscv.best_score_, gscv.best_estimator_

In [ ]:
Logis1 = gscv.best_estimator_

In [ ]:
logis1.fit(xtrain,ytrain)

In [ ]:
prediction = logis1.predict(xtest)

In [ ]:
print(classification_report(ytest, prediction))

In [ ]:
accuracy_score(ytest,prediction)

In [ ]:
metrics.plot_roc_curve(logis1,xtest, ytest)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rdf = RandomForestClassifier(n_estimators=100)

In [ ]:
params_rdf = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

In [ ]:
gscv2 = GridSearchCV(rdf, param_grid=params_rdf, return_train_score=True)

In [ ]:
gscv2.fit(xtrain,ytrain)

In [ ]:
rdf.fit(xtrain,ytrain)

In [ ]:
gscv2.best_params_, gscv2.best_score_ ,gscv2.best_estimator_

In [ ]:
rdf1 = gscv2.best_estimator_

In [ ]:
rdf1.fit(xtrain,ytrain)

In [ ]:
pred_rdf= rdf1.predict(xtest)

In [ ]:
accuracy_score(ytest,pred_rdf)

## Perceptron

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
ytrain.value_counts()

In [ ]:
ytrain_p = pd.get_dummies(ytrain)
ytrain_p = ytrain_p.drop(['death'], axis=1)
ytest_p = pd.get_dummies(ytest)
ytest_p = ytest_p.drop(['death'], axis=1)

In [ ]:
ytrain_p

In [ ]:
model = tf.keras.Sequential()

In [ ]:
model.add(layers.Input(xtrain.shape[1]))
model.add(layers.Dense(112, activation='relu'))
model.add(layers.Dense(50 , activation='relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
early_stopp = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10,
restore_best_weights=True)

In [ ]:
model.compile(optimizer='ADAM', loss='binary_crossentropy', metrics='accuracy')

In [ ]:
model.summary()

In [ ]:
hist = model.fit(xtrain, ytrain_p, epochs=100, batch_size=500, validation_data = (xtest, ytest_p))

In [ ]:
plt.plot(hist.history['loss'], '--', label='loss')
plt.plot(hist.history['val_loss'], '--', label='val_loss')
plt.legend()

In [ ]:
pre_perc = model.predict(xtest)

In [ ]:
result = np.where(pre_perc>=0.5,1,0)

In [ ]:
print(classification_report(ytest_p, result))

## save

In [ ]:
import pickle

In [ ]:
pickle.dump(logis1,open('logis.pkl','wb'))
pickle.dump(rdf1,open('rdf1.pkl','wb'))

In [ ]:
#aaa = pickle.load(open('logis.pkl','rb'))

In [ ]:
model.save('permodel.h5')